## Add in School Districts
Taking the school quality data, we will import GeoJSON file of the King County School Districts boundaries and attach each school to a school district

In [37]:
# import packages
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

In [38]:
# import necessary data
school_data = pd.read_csv('Cleaned-Data/merged-interpolated.csv')
district_geojson = gpd.read_file('Original-Data/Geospatial/School Districts King County.geojson')

In [39]:
# function to take school lat/long and return home district
def find_district(row):
    lat = row['GeoCoded_X']
    lon = row['GeoCoded_Y']
    
    # create point
    point = Point(lat, lon)
    
    # Check which polygon(s) contain the point
    containing_polygons = district_geojson[district_geojson.geometry.contains(point)]
    
    if containing_polygons.empty:
        return None
    else:
        # save district
        district = containing_polygons.iloc[0].to_dict()
        
        # Extract desired fields from the district
        district_name = district['NAME']
        district_id = district['DSTNUM']
        
        # Return a Series with the new columns
        return pd.Series({'District_Name': district_name, 'District_Num': district_id})

In [40]:
 # Apply the function and assign the result to new columns
school_data[['District_Name', 'District_Num']] = school_data.apply(find_district, axis=1)

print(school_data.head())

  SchoolYear  QualityScore                             School  GeoCoded_Y  \
0    2014-15      0.685667  Adams Elementary School (Seattle)   47.673457   
1    2015-16      0.569750  Adams Elementary School (Seattle)   47.673457   
2    2016-17      0.609750  Adams Elementary School (Seattle)   47.673457   
3    2017-18      0.397000  Adams Elementary School (Seattle)   47.673457   
4    2018-19      0.361857  Adams Elementary School (Seattle)   47.673457   

   GeoCoded_X     Grade Category District_Name District_Num  
0   -122.3918  Elementary School       Seattle        17001  
1   -122.3918  Elementary School       Seattle        17001  
2   -122.3918  Elementary School       Seattle        17001  
3   -122.3918  Elementary School       Seattle        17001  
4   -122.3918  Elementary School       Seattle        17001  


In [41]:
# save new file as csv
school_data.to_csv('Cleaned-Data/scores-districts.csv', index=False)